<a href="https://colab.research.google.com/github/dataplusplus-ai/Data-Science/blob/master/code_project_tensorflow__challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
wine_names = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline']
wine_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', names = wine_names)
wine_df = pd.DataFrame(wine_data)

# New Section

In [ ]:
wine_df.describe()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [ ]:
import tensorflow as tf

const1 = tf.constant([[1,2,3], [1,2,3]]);
const2 = tf.constant([[3,4,5], [3,4,5]]);

result = tf.add(const1, const2);

with tf.Session() as sess:
  output = sess.run(result)
  print(output)

[[4 6 8]
 [4 6 8]]



By Dante Sblendorio

What is a neural net, and how can you create one? Keep reading to find out. Below, I explain the basics of setting up a neural net using TensorFlow.

A neural net consists of three key components: the input layer, the hidden layer(s), and the output layer. Each layer has an arbitrary number of nodes (or neurons). In the example in the previous section, the input layer is const1 and const2. The matrix addition can be thought of as the hidden layer, and the output layer is the output. In the case of our wine data, the input data is the 13 chemical features and the output layer is the Class. The hidden layer can be thought of as a sophisticated ensemble of mathematical functions that behave as filters, extracting the relevant features for determining the correct Class. The structure of the neural net is inspired by biology, specifically the neural connections in the human brain. For the sake of this article, I won’t go into depth about the mathematical structure of the hidden layer(s). It is sufficient to think of it as a mathematical “black box” that extracts hidden meaning from the data. (However, if you want to learn more, this is a thorough online textbook on neural networks and deep learning.)

We split the dataset into a training and test set. This allows us to “train” the mathematical operators within the hidden layer to converge on ideal values that correctly predict the correct Class based on the 13 features for each observation. We then inject the test set into the neural net and evaluate the accuracy to determine how well the net has been trained. Splitting the data in this way provides a way to avoid overfitting or underfitting the data, thereby giving a true estimation of the accuracy of the net. In order to prepare the data for TensorFlow, we perform some slight manipulation:



In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

#this prepares our Class variable for the NN
def convertClass(val):
   if val == 1:
       return [1, 0, 0]
   elif val == 2:
       return [0, 1, 0]
   else:
       return [0, 0, 1]
   
Y = wine_df.loc[:,'Class'].values
Y = np.array([convertClass(i) for i in Y])
X = wine_df.loc[:,'Alcohol':'Proline'].values

#we split the dataset into a test and training set
train_x, test_x, train_y, test_y = train_test_split(X,Y , test_size=0.3, random_state=0)
train_x = train_x.transpose()
train_y = train_y.transpose()
test_x = test_x.transpose()
test_y = test_y.transpose()

Now that the data is prepped, we define several functions that form the foundation of the neural net. First, we define a function that establishes the initial parameters of our net. Here we also define how many nodes are in each of the hidden layers (I chose to have two hidden layers). Since we have three possible values for Class, we have three nodes in the output layer. Next we define a forward propagation function. All this does is send the 13 features through the net, and subject them to the mathematical operations within the hidden layer.

We also need to define a cost function. This is a critical function that allows us to “train” the network. It is a single value that describes how well the net does at predicting the correct Class. If the cost value is high, the mathematical operates adjust, and the data is fed through the net again. The data is fed through the net until the cost value converges.

In [ ]:
def init_parameters(num_input_features):

   num_hidden_layer =  32
   num_hidden_layer_2 = 16
   num_output_layer_1 = 3
   
   tf.set_random_seed(1)
   W1 = tf.get_variable("W1", [num_hidden_layer, num_input_features], initializer = tf.contrib.layers.xavier_initializer(seed=1))
   b1 = tf.get_variable("b1", [num_hidden_layer, 1], initializer = tf.zeros_initializer())
   W2 = tf.get_variable("W2", [num_hidden_layer_2, num_hidden_layer], initializer = tf.contrib.layers.xavier_initializer(seed=1))
   b2 = tf.get_variable("b2", [num_hidden_layer_2, 1], initializer = tf.zeros_initializer())
   W3 = tf.get_variable("W3", [num_output_layer_1, num_hidden_layer_2], initializer = tf.contrib.layers.xavier_initializer(seed=1))
   b3 = tf.get_variable("b3", [num_output_layer_1, 1], initializer = tf.zeros_initializer())
   
   parameters = {"W1": W1,
                 "b1": b1,
                 "W2": W2,
                 "b2": b2,
                 "W3": W3,
                 "b3": b3}
   
   return parameters
			
def for_prop(X, parameters):
			
   W1 = parameters['W1']
   b1 = parameters['b1']
   W2 = parameters['W2']
   b2 = parameters['b2']
   W3 = parameters['W3']
   b3 = parameters['b3']
   
   # propagates values through NN using Rectified Linear Unit as activation function          
   Z1 = tf.add(tf.matmul(W1, X), b1)                     
   A1 = tf.nn.relu(Z1)                                    
   Z2 = tf.add(tf.matmul(W2, A1), b2)                     
   A2 = tf.nn.relu(Z2)                                   
   Z3 = tf.add(tf.matmul(W3, A2), b3)                    
   return Z3

def c(Z3, Y):
   logits = tf.transpose(Z3)
   labels = tf.transpose(Y)
   cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
   return cost

We also need to define a function that produces a random subset of observations within the training set. I mentioned previously that data is fed into the neural net until the cost value converges; each iteration a random subsample is picked out of the total training set and injected into the net. In this function we create batches of subsamples:

In [ ]:
def rand_batches(X, Y, batch_size = 128, seed = 0):
   m = X.shape[1]
   batches = []
   np.random.seed(seed)
   
   # shuffle
   permutation = list(np.random.permutation(m))
   shuffled_X = X[:, permutation]
   shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

   # partition the shuffled data
   num_batches = math.floor(m/batch_size)
   for k in range(0, num_batches):
       batch_X = shuffled_X[:, k * batch_size : k * batch_size + batch_size]
       batch_Y = shuffled_Y[:, k * batch_size : k * batch_size + batch_size]
       batch = (batch_X, batch_Y)
       batches.append(batch)
   
   # handle end case
   if m % batch_size != 0:
       batch_X = shuffled_X[:, num_batches * batch_size : m]
       batch_Y = shuffled_Y[:, num_batches * batch_size : m]
       batch = (batch_X, batch_Y)
       batches.append(batch)
   
   return batches

In [ ]:
member_number =  12438336

one = [member_number, int(member_number/5), int(member_number/100)]

two = [0.02, 0.05, 0.08]

a = tf.placeholder(tf.float32, shape=(3))

b = tf.placeholder(tf.float32, shape=(3))

result = tf.tensordot(a, b, 1)

with tf.Session() as sess:

   print(int(result.eval(feed_dict={a: one, b: two})))

383100


Running the net

By Dante Sblendorio

In the preceding sections of this contest, we’ve gone through all the steps of installing TensorFlow, setting up a Jupyter Notebook and creating a neural net. Now it’s time for the real fun: Running the neural net and viewing our results. Keep reading for this last and final step in our journey.

Now that we have defined all the functions we need, we can construct the neural net. We first initialize all the variables and parameters based on the shape of the training set, and then define the learning rate, number of epochs, and batch size. The learning rate determines how fast the mathematical operators converge on the minimum cost value. The number of epochs is the number of times training data is fed through the net. The batch size is the size of each random subsample. Each parameter has a role in the final test accuracy and how fast the net converges.

In [ ]:
from tensorflow.python.framework import ops
import math

def nn(train_x, train_y, test_x, test_y, learning_rate ,num_epochs, batch_size, print_cost = True):
       ops.reset_default_graph()
       tf.set_random_seed(1)
       (n_x, m) = train_x.shape
       n_y = train_y.shape[0]
       
       # Initialize
       costs = []
       X = tf.placeholder(tf.float32, [n_x, None], name="X")
       Y = tf.placeholder(tf.float32, [n_y, None], name="Y")
       parameters = init_parameters(13)
       Z3 = for_prop(X, parameters)
       cost = c(Z3, Y)
       optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
       init = tf.global_variables_initializer()

       # Forward propagation
       with tf.Session() as sess:
           sess.run(init)

           #training loop
           for epoch in range(num_epochs):
               epoch_cost = 0.
               num_batches = int(m / batch_size)
               batches = rand_batches(train_x, train_y, batch_size, 13)
               for batch in batches:
                   (batch_X, batch_Y) = batch
                   _ , batch_cost = sess.run([optimizer, cost], feed_dict={X: batch_X, Y: batch_Y})
                   epoch_cost += batch_cost / num_batches

               # print the cost at every 50 epochs
               if print_cost == True and epoch % 50 == 0:
                   print ("Epoch %i cost: %f" % (epoch, epoch_cost))
               if print_cost == True and epoch % 5 == 0:
                   costs.append(epoch_cost)

           # Save the parameters
           parameters = sess.run(parameters)
           print("Parameters trained...")

           # Calculate the correct predictions
           correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

           # Accuracy of the test set
           accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

           print("Train Accuracy:", accuracy.eval({X: train_x, Y: train_y}))
           print("Test Accuracy:", accuracy.eval({X: test_x, Y: test_y}))

           return parameters

Now that we have defined the neural net function, we can pick values for each parameter, and run:

In [ ]:
learning_rate = 0.001 #change this to change learning rate
num_epochs = 1000     #change this to change number of epochs
batch_size = 20       #change this to change batch size
parameters = nn(train_x, train_y, test_x, test_y, learning_rate ,num_epochs, batch_size)

Epoch 0 cost: 35.883492
Epoch 50 cost: 0.522927
Epoch 100 cost: 0.311308
Epoch 150 cost: 0.240480
Epoch 200 cost: 0.200274
Epoch 250 cost: 0.171992
Epoch 300 cost: 0.146479
Epoch 350 cost: 0.142560
Epoch 400 cost: 0.124002
Epoch 450 cost: 0.114809
Epoch 500 cost: 0.107478
Epoch 550 cost: 0.101389
Epoch 600 cost: 0.096205
Epoch 650 cost: 0.091718
Epoch 700 cost: 0.087791
Epoch 750 cost: 0.084321
Epoch 800 cost: 0.081231
Epoch 850 cost: 0.078456
Epoch 900 cost: 0.075943
Epoch 950 cost: 0.073653
Parameters trained...
Train Accuracy: 0.9919355
Test Accuracy: 0.962963


In [ ]:
learning_rate = 0.0001 #change this to change learning rate
num_epochs = 1000     #change this to change number of epochs
batch_size = 20       #change this to change batch size
parameters = nn(train_x, train_y, test_x, test_y, learning_rate ,num_epochs, batch_size)

Epoch 0 cost: 49.291391
Epoch 50 cost: 1.906928
Epoch 100 cost: 1.459944
Epoch 150 cost: 1.108970
Epoch 200 cost: 0.878475
Epoch 250 cost: 0.723207
Epoch 300 cost: 0.610176
Epoch 350 cost: 0.528843
Epoch 400 cost: 0.468302
Epoch 450 cost: 0.421824
Epoch 500 cost: 0.380954
Epoch 550 cost: 0.348661
Epoch 600 cost: 0.321599
Epoch 650 cost: 0.295100
Epoch 700 cost: 0.274693
Epoch 750 cost: 0.257200
Epoch 800 cost: 0.239150
Epoch 850 cost: 0.226493
Epoch 900 cost: 0.213283
Epoch 950 cost: 0.203568
Parameters trained...
Train Accuracy: 0.9354839
Test Accuracy: 0.9444444


With the learning rate, number of epochs, and batch size defined as they are, the neural net does a pretty good job with the test set, accurately predicting the Class more than 96% of the time. Try changing the values of each parameter and observe how the accuracies change (it is also a good exercise to do to better understand how the neural net functions). For more on learning rate, epochs, and batch size, there is a great article to read here.

To generate an entry code for Challenge 5, create a new code cell in your Jupyter notebook and paste the following code in it:



In [ ]:
member_number = 12438336

one = [member_number, int(member_number/5), int(member_number/25), int(member_number/50), int(member_number/100)]

two = [0.02, 0.05, 0.08, 0.11, 0.14]

a = tf.placeholder(tf.float32, shape=(5))

b = tf.placeholder(tf.float32, shape=(5))

result = tf.tensordot(a, b, 1)

with tf.Session() as sess:

   print(int(result.eval(feed_dict={a: one, b: two})))

457730
